In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
strategy = tf.distribute.get_strategy()
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/train.xlsx")
test = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/evaluation.xlsx")

# Tokenisation

In [ ]:
from transformers import TFAutoModel,AutoTokenizer, AutoModel
import tensorflow as tf

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')

In [ ]:
train_enc=tokenizer.batch_encode_plus(train[['text','reason']].values.tolist(),padding='max_length',max_length=100,truncation=True,return_attention_mask=True)
test_enc=tokenizer.batch_encode_plus(test[['text','reason']].values.tolist(),padding='max_length',max_length=100,truncation=True,return_attention_mask=True)
train_tf1=tf.convert_to_tensor(train_enc['input_ids'],dtype=tf.int32)
train_tf2=tf.convert_to_tensor(train_enc['attention_mask'],dtype=tf.int32)
train_input={'input_word_ids':train_tf1,'input_mask':train_tf2}
test_tf1=tf.convert_to_tensor(test_enc['input_ids'],dtype=tf.int32)
test_tf2=tf.convert_to_tensor(test_enc['attention_mask'],dtype=tf.int32)
test_input={'input_word_ids':test_tf1,'input_mask':test_tf2}

# Model Definition

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True)
model.fit(train_input,train.label,validation_split = 0.2,epochs=2,batch_size=16*strategy.num_replicas_in_sync,callbacks=[early_stop],verbose=1)

Epoch 1/2


1917/2423 [======================>.......] - ETA: 10:35 - loss: 0.6265 - accuracy: 0.5732

In [ ]:
#x = np.asarray(test).astype('float32')

In [ ]:
#test_input=bert_encode(test.hypothesis.values,test.premise.values,tokenizer)
#predictions = [np.argmax(i) for i in model.predict(test_input)]

# Inference

In [ ]:
pred = model.predict(test_input)

In [ ]:
predx = [list(i).index(max(list(i))) for i in list(pred)]
predx

In [ ]:
test['pred'] = predx

In [ ]:
test.to_csv("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/pred_evalantsyn.csv", index=False)

In [5]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/pred_evalantaug.csv")

In [6]:
test

,text,reason,label,pred
0,the app is crashing when i play a vedio,app crashes during playback,1.0,1
1,but i want to connect it to the tv from one de...,want compatibility with more smart televisions,0.0,1
2,very helpful when and home working remotley,good app for work,0.0,1
3,this zoom so called and missed call and mobile...,receiving incorrect phone number message,0.0,1
4,one of my favorite apps,good for spending time,0.0,1
...,...,...,...,...
8995,very good streaming platform has a lot of content,app has very good content,1.0,1
8996,so paying for an app that is not fit for purpose.,unable to use app,0.0,1
8997,"it does not work well for me, it loads, but if...",app keeps loading indefinitely,0.0,1
8998,i can not watch the new disney shows.,unable to load the episodes list,0.0,1


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy_score(test['pred'], test['label'])

0.35988888888888887

In [12]:
recall_score(test['pred'], test['label'])

0.33313180169286577

In [13]:
precision_score(test['pred'], test['label'])

0.9180273242252582

In [14]:
f1_score(test['pred'], test['label'])

0.48886522934965837

In [ ]:
test['pred']

In [ ]:
test['label']

In [10]:
len(test[test['pred'] == 1])/len(test), len(test[test['pred'] == 0])/len(test)

(0.9188888888888889, 0.0811111111111111)